In [1]:
!pip install --upgrade pip
!pip install faiss-cpu sentence-transformers transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 14.6 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 23.3.1
    Uninstalling pip-23.3.1:
      Successfully uninstalled pip-23.3.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 39.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 50.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 48.7 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.7/781.7 kB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 53.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.7/37.7 MB 55.0 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.4.0
    Uninstalling typing_extensions-4.4.0:
      Successfully uninstalled typi

In [5]:
!pip uninstall -y numpy
!pip install numpy==1.24.4

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 24.2 MB/s eta 0:00:00a 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
faiss-cpu 1.11.0 requires numpy<3.0,>=1.25.0, but you have numpy 1.24.4 which is incompatible.


In [1]:
import json
import faiss
import numpy as np
import re
from sentence_transformers import SentenceTransformer
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# ✅ OpenChat 모델 로딩
model_id = "openchat/openchat-3.5-0106"

tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype="auto"
)

chat = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=768,
    temperature=0.7,
    do_sample=True
)

# ✅ game.json 로딩
with open("game.json", "r", encoding="utf-8") as f:
    data = json.load(f)

texts = [item["text"] for item in data]
game_names = [item["game_name"] for item in data]

# ✅ 임베딩 모델 로딩
embed_model = SentenceTransformer("BAAI/bge-m3", device="cuda")
embeddings = embed_model.encode(
    texts,
    normalize_embeddings=True,
    batch_size=8,
    show_progress_bar=True
)

# ✅ FAISS 인덱스 구축
dimension = embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)
index.add(np.array(embeddings))

# ✅ 출력 후처리 함수
def clean_output(raw_output: str) -> str:
    # assistant 토큰 이후만 추출
    if "<|assistant|>" in raw_output:
        raw_output = raw_output.split("<|assistant|>")[-1]

    # "추천 완료!" 이전까지만 유지
    if "추천 완료!" in raw_output:
        raw_output = raw_output.split("추천 완료!")[0]

    return raw_output.strip()

# ✅ 추천 함수
def recommend_with_openchat(query, default_k=3):
    number_match = re.search(r'(\d+)\s*개', query)
    top_k = int(number_match.group(1)) if number_match else default_k

    # 유사도 검색
    query_embedding = embed_model.encode([query], normalize_embeddings=True)
    D, I = index.search(np.array(query_embedding), k=top_k)

    # 검색 결과 → 프롬프트 구성용
    retrieved = []
    selected_names = []
    for i in I[0]:
        name = game_names[i]
        desc = texts[i]
        retrieved.append(f"[{name}]\n{desc}")
        selected_names.append(f"- {name}")

    context = "\n\n".join(retrieved)
    name_list_str = "\n".join(selected_names)

    # 프롬프트
    prompt = f"""아래는 보드게임 설명입니다. 각 게임은 "[게임명]\n설명" 형식으로 되어 있습니다.

[게임 설명]
{context}

⚠️ 반드시 아래의 게임 이름 목록 중에서만 선택할 수 있습니다. 이 목록에 없는 게임 이름을 절대 생성하지 마세요.  
만약 목록에 없는 게임명을 출력하면 실패로 간주됩니다.

[허용된 게임 이름 목록]
{ name_list_str }

[사용자 질문]
{query}

📌 출력 지침:
- 반드시 위 목록에 있는 게임 중에서만 3개를 골라 추천하세요.
- 출력 형식은 반드시 아래 형식처럼 작성하세요:

게임명1: 추천 이유  
게임명2: 추천 이유  
게임명3: 추천 이유

- 각 줄은 '게임명: 추천 이유' 형식으로만 작성하고, 줄바꿈 이외에 아무 포맷도 쓰지 마세요.
- 추천이 모두 끝나면 마지막 줄에 반드시 다음과 같이 써주세요:  
추천 완료!

그 이후에는 아무 것도 쓰지 마세요.
"""

    chat_prompt = f"<|system|>\n너는 보드게임 추천 도우미야. 사용자의 질문에 따라 관련 게임을 추천하고 이유도 알려줘.\n<|user|>\n{prompt}\n<|assistant|>"
    raw_output = chat(chat_prompt)[0]["generated_text"]
    cleaned = clean_output(raw_output)

    print("\n🤖 추천게임 (OpenChat 기반):\n")
    print(cleaned)




Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Device set to use cuda:0
/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should o

Batches:   0%|          | 0/29 [00:00<?, ?it/s]

In [2]:
# ✅ 실행
if __name__ == "__main__":
    print("🎲 보드게임 추천 시스템 (OpenChat 기반)")
    user_query = input("❓ 어떤 게임을 원하시나요?\n예시: '전략적이고 빠른 게임 3개 추천해줘'\n> ")
    recommend_with_openchat(user_query)

🎲 보드게임 추천 시스템 (OpenChat 기반)


❓ 어떤 게임을 원하시나요?
예시: '전략적이고 빠른 게임 3개 추천해줘'
>  전략적이고 빠른 게임 3개 추천해줘



🤖 추천게임 (OpenChat 기반):

게임명1: 쇼텐토텐  
추천 이유: 쇼텐토텐은 전략적으로 생각하고 빠른 게임입니다. 게임의 목표는 자신이 속한 팀이 상대 팀에게 가장 많은 돌을 빼아 가는 것입니다. 각 턴에 돌을 빼는 방법은 다양하게 생각해야 하며, 전략을 세우고 실행하는 것이 중요합니다.

게임명2: 언락  
추천 이유: 언락은 전략적이고 빠른 게임으로, 시나리오 종류마다 다른 이야기로 만들어졌습니다. 게임의 목표는 선택한 시나리오에 따라 다르지만, 대부분의 경우 자신의 팀이 목표를 달성하는 것이 중요합니다. 게임 진행에 있어 전략을 세우고 실행하는 것이 필요하며, 빠르게 가상 세상에서 이야기를 만들어 나갑니다.

게임명3: 뱅  
추천 이유: 뱅은 전략적이고 빠른 게임으로, 무법자, 부관, 배신자, 보안관 중 하나가 되어 목표를 이루는 것이 게임의 목표입니다. 게임 진행에 있어 다른 플레이어들의 행동을 추적하고 예측하는 것이 중요하며, 전략을 세우고 실행하는 것이 필요합니다. 빠른 속도로 게임을 진행하면서도 전략을 세우고 실행하는 것이 중요하다는 점을 보여줍니다.
